- Daten einlesen
- Texte bereinigen
- Modell(e) trainieren
- Accuracy, Precision & Recall berechnen


### Schritt 1A: Einlesen der Rohdaten
In diesem Schritt wird geprüft, ob alle Urteilstexte aus dem bereitgestellten OpenJur-Datensatz korrekt geladen werden können. Ziel ist es, die Datengrundlage zu verifizieren, bevor weitere Verarbeitungsschritte erfolgen.

In [6]:
import os
import pandas as pd
import re

DATA_DIR = "../data/Gerichtsurteile_Openjur" 
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".txt")]

print("Pfad:", os.path.abspath(DATA_DIR))
print("Anzahl .txt:", len(files))
print("Erste 10 Dateien:", files[:10])


Pfad: c:\Users\Lena\Documents\GitHub\ds_law\backend\data\Gerichtsurteile_Openjur
Anzahl .txt: 2375
Erste 10 Dateien: ['2090187.txt', '2112111.txt', '2112115.txt', '2112117.txt', '2112118.txt', '2112119.txt', '2112121.txt', '2112123.txt', '2124977.txt', '2126821.txt']


Die Ausgabe bestätigt, dass insgesamt 2375 Urteilstexte erfolgreich eingelesen wurden. Die Dateinamen entsprechen den von OpenJur vergebenen Fall-IDs, wodurch die Konsistenz und Vollständigkeit der Datengrundlage sichergestellt ist.

### Schritt 1B: Filterung nach LG + Extraktion der Variablen
Aufgabe: Filterung nach LG muss noch in den Prompt integriert werden

In [ ]:
import sys
import os
import re
import json
import pandas as pd
import google.generativeai as genai

# --- 1. SETUP & EINLESEN ---
# Hinweis: DATA_DIR und files müssen in deiner Umgebung definiert sein
rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    with open(os.path.join(DATA_DIR, fn), "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    rows.append({"case_id": case_id, "text": text})

df = pd.DataFrame(rows)

# --- 2. TEXT-EXTRAKTION ---
df["head"] = df["text"].str.slice(0, 8000)

def extract_tenor(text: str) -> str:
    if not isinstance(text, str): return ""
    m_start = re.search(r"\bTenor\b", text, flags=re.IGNORECASE)
    if not m_start: return ""
    start = m_start.end()
    m_end = re.search(r"\b(Tatbestand|Gründe|Gruende|Entscheidungsgründe|Entscheidungsgruende)\b", 
                      text[start:], flags=re.IGNORECASE)
    end = start + m_end.start() if m_end else min(len(text), start + 8000)
    return text[start:end].strip()

df["tenor"] = df["text"].apply(extract_tenor)

# --- 3. LG-FILTER (REGEX) ---
# Fokus auf Landesgerichtsurteile gemäß Aufgabenstellung [cite: 21]
lg_pattern = r"\bLandgericht\b|\bLG\s+[A-ZÄÖÜa-zäöü]+"
df["is_landgericht"] = df["head"].str.contains(lg_pattern, case=False, regex=True, na=False)
df_lg = df[df["is_landgericht"] == True].copy()

print("-" * 40)
print(f"STATISTIK")
print(f"Gesamt eingelesen: {len(df)}")
print(f"Gefilterte LGs:    {len(df_lg)}")
print("-" * 40)

# --- 4. DEFINITION DES UMFASSENDEN PROMPTS ---
COMPREHENSIVE_PROMPT = """
Lies den Text genau und analysiere das vorliegende Gerichtsurteil. Extrahiere die folgenden Informationen präzise. Falls eine Information im Text nicht auffindbar ist, gib "null" an.

### Extraktions-Anweisungen:

1. **Input-Variablen (Features):**
    * **Dieselmotor_Typ (String):** Welcher Motortyp (z. B. EA 189, EA 288)?
    * **Art_Abschalteinrichtung (String):** Beschreibung der genannten Abschalteinrichtung.
    * **KBA_Rueckruf (Boolean):** Verpflichtender Rückruf des Kraftfahrt-Bundesamtes? (true/false)
    * **Fahrzeugstatus (String):** "Neuwagen" oder "Gebrauchtwagen"?
    * **Fahrzeugmodell_Baureihe (String):** Bezeichnung des Modells.
    * **Update_Status (Boolean/null):** Software-Update aufgespielt? (true/false/null)
    * **Kilometerstand_Kauf (Integer):** Stand bei Erwerb.
    * **Kilometerstand_Klageerhebung (Integer):** Stand bei Klageeinreichung.
    * **Erwartete_Gesamtlaufleistung (Integer):** Vom Gericht angenommene Gesamtlaufleistung.
    * **Kaufdatum (Date):** Format: YYYY-MM-DD.
    * **Uebergabedatum (Date):** Format: YYYY-MM-DD.
    * **Datum_Klageerhebung (Date):** Format: YYYY-MM-DD.
    * **Nachweis_Aufklaerung (Boolean):** Gab es eine Anlage zum Kaufvertrag über die Software? (true/false)
    * **Beklagten_Typ (String):** "Händler" oder "Hersteller".
    * **Datum_Urteil (Date):** Format: YYYY-MM-DD.
    * **Kaufpreis (Float):** Betrag in Euro (ohne Zinsen).
    * **Nacherfuellungsverlangen_Fristsetzung (String):** "Ja", "Nein", "Entbehrlich".
    * **Klageziel (String):** z. B. "Rückabwicklung", "Minderung", "Schadensersatz".
    * **Rechtsgrundlage (String):** z. B. § 437 BGB oder § 826 BGB.

2. **Zielvariablen (Labeling):**
    * **LABEL_Anspruch_Schadensersatz (Boolean):** Kläger erhält Schadensersatz? (true/false) [cite: 27, 28]
    * **LABEL_Schadensersatzhoehe_Betrag (Float):** Zugesprochener Betrag in Euro (ohne Zinsen). [cite: 30]
    * **LABEL_Schadensersatzhoehe_Range (String):** Einordnung: "< 5000", "5000-10000", "10001-15000", "> 15000", "Klage abgewiesen".

3. **Kategorisierung "Sonstige":**
    Sollte im Tenor nichts über Abweisung oder Schadensersatz stehen (sondern Streitwertfestsetzung, Ablehnungsgesuche etc.), kategorisiere als "Sonstige"[cite: 33, 34]. Gib im Feld "Urteil_Anmerkung" die Begründung an.

### Ausgabeformat:
Antworte AUSSCHLIESSLICH als JSON-Liste mit einem Eintrag. Ignoriere Streitwerte und Zinsen.
"""

# --- 5. TEST: EINZELNES URTEIL PRÜFEN ---
# hier einfügen

if not df_lg.empty:
    test_row = df_lg.iloc[0]
    test_input = f"Kopf: {test_row['head']}\nTenor: {test_row['tenor']}\n\n{COMPREHENSIVE_PROMPT}"
    
    print(f"Teste Extraktion für Case: {test_row['case_id']}...")
    response = model.generate_content(test_input)
    print("\nErgebnis des Einzeltests:")
    print(response.text)
else:
    print("Keine LG-Urteile zum Testen gefunden.")

# --- 6. BATCH-DATEI ERSTELLEN (AKTUELL AUSKOMMENTIERT) ---
"""
batch_file_path = "gemini_batch_input_lg_comprehensive.jsonl"
with open(batch_file_path, "w", encoding="utf-8") as f:
    for _, row in df_lg.iterrows():
        payload = {
            "custom_id": f"case_{row['case_id']}",
            "contents": [{
                "role": "user",
                "parts": [{"text": f"Kopf: {row['head']}\nTenor: {row['tenor']}\n\n{COMPRE_PROMPT}"}]
            }]
        }
        f.write(json.dumps(payload) + "\n")

# BATCH-JOB STARTEN
print("Lade Batch-Datei hoch...")
input_file = genai.upload_file(path=batch_file_path, mime_type="application/jsonl")
batch_job = genai.create_batch_job(
    model="models/gemini-1.5-pro",
    input_file=input_file.name,
    output_file_name="ergebnisse_diesel_detailed"
)
print(f"Batch-Job gestartet: {batch_job.name}")
"""

----------------------------------------
STATISTIK
Gesamt eingelesen: 2375
Gefilterte LGs:    2088
----------------------------------------
Teste Extraktion für Case: 2090187...


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
Please retry in 28.4536242s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generat

In [17]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f"Modell-Name: {m.name}")

Modell-Name: models/gemini-2.5-flash
Modell-Name: models/gemini-2.5-pro
Modell-Name: models/gemini-2.0-flash-exp
Modell-Name: models/gemini-2.0-flash
Modell-Name: models/gemini-2.0-flash-001
Modell-Name: models/gemini-2.0-flash-lite-001
Modell-Name: models/gemini-2.0-flash-lite
Modell-Name: models/gemini-2.0-flash-lite-preview-02-05
Modell-Name: models/gemini-2.0-flash-lite-preview
Modell-Name: models/gemini-exp-1206
Modell-Name: models/gemini-2.5-flash-preview-tts
Modell-Name: models/gemini-2.5-pro-preview-tts
Modell-Name: models/gemma-3-1b-it
Modell-Name: models/gemma-3-4b-it
Modell-Name: models/gemma-3-12b-it
Modell-Name: models/gemma-3-27b-it
Modell-Name: models/gemma-3n-e4b-it
Modell-Name: models/gemma-3n-e2b-it
Modell-Name: models/gemini-flash-latest
Modell-Name: models/gemini-flash-lite-latest
Modell-Name: models/gemini-pro-latest
Modell-Name: models/gemini-2.5-flash-lite
Modell-Name: models/gemini-2.5-flash-image
Modell-Name: models/gemini-2.5-flash-preview-09-2025
Modell-Name:

In [ ]:
'''
import sys
!{sys.executable} -m pip install -U google-generativeai
import os
import re
import json
import pandas as pd
import google.generativeai as genai

# --- 1. SETUP & EINLESEN ---
rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    with open(os.path.join(DATA_DIR, fn), "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    rows.append({"case_id": case_id, "text": text})

df = pd.DataFrame(rows)

# --- 2. TEXT-EXTRAKTION ---
df["head"] = df["text"].str.slice(0, 8000)

def extract_tenor(text: str) -> str:
    if not isinstance(text, str): return ""
    m_start = re.search(r"\bTenor\b", text, flags=re.IGNORECASE)
    if not m_start: return ""
    start = m_start.end()
    m_end = re.search(r"\b(Tatbestand|Gründe|Gruende|Entscheidungsgründe|Entscheidungsgruende)\b", 
                      text[start:], flags=re.IGNORECASE)
    end = start + m_end.start() if m_end else min(len(text), start + 8000)
    return text[start:end].strip()

df["tenor"] = df["text"].apply(extract_tenor)


# --- 3. DEIN PRÄZISER LG-FILTER ---
# Wir nutzen dein Muster und filtern den Dataframe sofort
lg_pattern = r"\bLandgericht\b|\bLG\s+[A-ZÄÖÜa-zäöü]+"
df["is_landgericht"] = df["head"].str.contains(lg_pattern, case=False, regex=True, na=False)

# Nur die Landgerichte in einen neuen Dataframe extrahieren
df_lg = df[df["is_landgericht"] == True].copy()

print("-" * 40)
print(f"STATISTIK (Dein Regex-Filter)")
print(f"Gesamt eingelesen: {len(df)}")
print(f"Gefilterte LGs:    {len(df_lg)}")
print("-" * 40)


# --- 4. BATCH-DATEI ERSTELLEN ---
PROMPT_TEXT = """
Analysiere den Text präzise:
1. Gerichtstyp: Bestimme das Gericht (NUR 'LG' oder 'OLG').
2. Geldbetrag oder Abweisung: Betrag in Euro oder 'Klage abgewiesen'.
3. Sonstige: 'Sonstige' bei Streitwertfestsetzungen.
Antworte NUR als JSON: {"Gerichtstyp": "LG/OLG", "Urteil": "Ergebnis"}
"""

batch_file_path = "gemini_batch_input_lg.jsonl"
with open(batch_file_path, "w", encoding="utf-8") as f:
    # Wir nehmen nur die gefilterten LGs!
    for _, row in df_lg.iterrows():
        payload = {
            "custom_id": f"case_{row['case_id']}",
            "contents": [{
                "role": "user",
                "parts": [{"text": f"Kopf: {row['head']}\nTenor: {row['tenor']}\n\n{PROMPT_TEXT}"}]
            }]
        }
        f.write(json.dumps(payload) + "\n")

# --- 5. BATCH-JOB STARTEN (MIT VERSION-FIX) ---
api_key = os.getenv("GEMINI_API_KEY")
if api_key:
    genai.configure(api_key=api_key)
    print("Lade Datei hoch...")
    input_file = genai.upload_file(path=batch_file_path, mime_type="application/jsonl")
    
    # Da create_batch_job bei dir einen Fehler warf: 
    # Wir nutzen das Modell-Objekt direkt, das ist oft kompatibler
    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        # Falls dein SDK die Methode noch nicht kennt, hilft nur das Update im Terminal!
        batch_job = genai.create_batch_job(
            model="models/gemini-1.5-flash",
            input_file=input_file.name,
            output_file_name="ergebnisse_diesel_final"
        )
        print(f"✓ Job erfolgreich gestartet! ID für Alina: {batch_job.name}")
    except AttributeError:
        print("✗ FEHLER: Dein Paket 'google-generativeai' ist zu alt.")
        print("FIX: Tippe 'pip install -U google-generativeai' ins Terminal und RESTARTE den Kernel!")
'''


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


----------------------------------------
STATISTIK (Dein Regex-Filter)
Gesamt eingelesen: 2375
Gefilterte LGs:    2088
----------------------------------------


Die resultierende Datenstruktur umfasst 2375 gerichtliche Entscheidungen. Jede Zeile entspricht einem Urteil, das eindeutig über eine Fall-ID referenziert ist. Zusätzlich wurde die Textlänge der Urteile berechnet, um eine Plausibilitätsprüfung der Datengrundlage zu ermöglichen.

### Schritt 3: Prompt zur Extrahierung der Zielvariablen
Filterung nochmal nach LG (damit wir alle abfangen, die wir vorher nicht abfangen konnten) + Training in Batches

### Schritt 1C: Einschränkung auf Entscheidungen der Landgerichte

Gemäß der Aufgabenstellung wird der Datensatz auf Urteile deutscher Landgerichte beschränkt. Hintergrund ist, dass unterschiedliche
Gerichtsebenen teils abweichende rechtliche Maßstäbe anwenden, was zu inkonsistenten Lernsituationen für das Modell führen kann. 
Zunächst wird heuristisch anhand typischer Gerichtsbezeichnungen im Kopfbereich der Entscheidung geprüft, ob es sich um ein Urteil eines deutschen Landgerichts handelt. Anschließend wird der thematische Bezug zum Dieselskandal anhand zentraler Schlüsselbegriffe identifiziert. Durch diese getrennte Markierung wird Transparenz über die Zusammensetzung des Datensatzes geschaffen und eine spätere Anpassung der Filterlogik ermöglicht, ohne frühzeitig Daten zu verwerfen.

In [ ]:
# lg_pattern = r"\bLandgericht\b|\bLG\s+[A-ZÄÖÜa-zäöü]+"

# df["is_landgericht"] = df["head"].str.contains(lg_pattern, case=False, regex=True)

# df["is_landgericht"].value_counts()


KeyError: 'head'

### Schritt 1D: Finaler Analyse-Datensatz 
Der finale Analyse-Datensatz umfasst ausschließlich dieselbezogene Landgerichtsurteile und dient als Grundlage für die weitere Modellierung.

In [ ]:
#df_analysis = df[df["is_diesel_case"] & df["is_landgericht"]].copy()
#df_analysis.shape


KeyError: 'is_diesel_case'

Zur methodischen Absicherung wurde überprüft, ob alle Dokumente entsprechende Schlüsselbegriffe enthalten. Die Analyse bestätigt, dass nahezu alle Urteile einen expliziten Bezug zum Dieselskandal aufweisen.

### Schritt 2: Definition der Zielvariable

Ziel der Analyse ist die Vorhersage, ob in einem Urteil ein Schadensersatz zugesprochen wurde oder nicht. 
Die Zielvariable wird binär modelliert (1 = Schadensersatz zugesprochen, 0 = kein Schadensersatz). 
Da strukturierte Labels nicht vorliegen, erfolgt die Ableitung regelbasiert anhand typischer Formulierungen im Tenor der Entscheidung.



In [ ]:
'''
def infer_target(tenor: str):
    if not isinstance(tenor, str) or tenor.strip() == "":
        return None

    t = tenor.lower()

    # Verurteilung zur Zahlung → Schadensersatz
    positive_patterns = [
        r"wird verurteilt.*\bzu zahlen\b",
        r"wird verurteilt.*\ban den kläger\b.*\bzu zahlen\b",
        r"wird verurteilt.*\ban die klägerin\b.*\bzu zahlen\b",
        r"\bzu zahlen\b.*(\bEUR\b|€)",
        r"(\bEUR\b|€).*?\bzu zahlen\b",
        r"schadensersatz"
    ]
    if any(re.search(p, t, flags=re.DOTALL) for p in positive_patterns):
        return 1

    # Klage abgewiesen → kein Schadensersatz
    negative_patterns = [
        "die klage wird abgewiesen",
        "klage wird abgewiesen",
        "die berufung wird zurückgewiesen",
        "berufung wird zurückgewiesen",
        "wird zurückgewiesen",
        "wird verworfen",
        "als unzulässig verworfen",
        "wird als unzulässig verworfen",
    ]
    if any(p in t for p in negative_patterns):
        return 0

    return None


df_analysis["target"] = df_analysis["tenor"].apply(infer_target)
df_analysis["target"].value_counts(dropna=False)
'''

target
0.0    1026
1.0     715
NaN     315
Name: count, dtype: int64

In [ ]:
# df_analysis[df_analysis["target"].notna()][["case_id","target","tenor"]].sample(20, random_state=42)



,case_id,target,tenor
704,2274949,0.0,1. Die Berufung der Klagepartei gegen das Urte...
2074,2453299,0.0,1. Die Klage wird abgewiesen.2. Die Kosten des...
1319,2343116,0.0,1. Die Berufung des Klägers gegen das Urteil d...
1396,2350176,0.0,Die Berufung der Klägerin gegen das am 30. Okt...
984,2306384,1.0,Auf die Berufung der Beklagten wird das am 31....
398,2205120,0.0,Die Klage wird abgewiesen.Die Klägerin trägt d...
636,2270509,0.0,Die Klage wird abgewiesen.Die Kosten des Recht...
1825,2394941,0.0,Die Berufung des Klägers gegen das am 16. Apri...
1586,2378432,0.0,1. Die Klage wird abgewiesen.2. Der Kläger hat...
858,2297195,0.0,1. Auf die Berufung der Beklagten wird das End...


Die regelbasierte Ableitung der Zielvariable wurde anhand einer zufälligen Stichprobe überprüft, wobei ausschließlich der jeweilige Tenor betrachtet wurde.

**Umgang mit unklaren Fällen**

Nicht alle Urteile enthalten eine eindeutig identifizierbare Tenorformulierung, aus der zweifelsfrei hervorgeht, ob ein Schadensersatz zugesprochen wurde oder nicht (z. B. bei Vergleichen oder rein prozessualen Entscheidungen).  
Diese Fälle werden in der Zielvariable als *unklar* (`None`) gekennzeichnet und für die weitere Modellierung ausgeschlossen.  
Durch diese Einschränkung wird sichergestellt, dass das Modell ausschließlich auf eindeutig gelabelten Entscheidungen trainiert wird und Verzerrungen durch unsichere Zuordnungen vermieden werden.

In [ ]:
# prompt 

